# Discord API

Why do this? At the time the nodejs discord API only supported the prior version and I wanted new command features, so I reimplemented it over REST like the documentation instructs. Now, I believe the project has caught up.


DONE: copy code from https://github.com/briancullinan2/quake3-discord-bot/tree/main/discordApi

DISCORD: https://discord.gg/taHjA7Cg


https://discord.com/oauth2/authorize?response_type=code&client_id=1335769252409380884&scope=applications.commands%20bot%20guilds%20guilds.join%20email%20identify%20messages.read&permissions=54596725809&redirect_uri=https://briancullinan.com




## Authorization

First the gateway is established, then authorized bot messages can take place.

This method creates a websocket connection to discord and provides a requestAuthQ that restablishes connections with the gateway before making REST calls.




### Authorize Discord


#### the code

how to authorize and connect to discord?

authorize discord?

discord authorize?

discord request?



In [ ]:
const {request} = require('gaxios')
const WebSocket = require('ws')
const {delay, wait, timeout} = importer.import('discord utilities')
const {
  gatewayIdentified, gatewayClose, gatewayMessage,
} = importer.import('discord gateway')
const {
  TOKEN, DEFAULT_API, DEFAULT_RATE
} = importer.import('discord configuration')

let ws = false
let wsConnecting = false
let previousRequest = 0

async function gatewayUrl() {
  // TODO: return the same result if queried less than 1 second ago
  // doesn't use requestAuthQ because that would create an infinite loop
  let result = await request({
    headers: {
      'Authorization': `Bot ${TOKEN}`
    },
    method: 'GET',
    url: `${DEFAULT_API}gateway/bot`
  })
  return result.data
}

function gatewayOpen() {
  console.log('Connecting to Discord')
}

async function authorizeGateway() {
  if(wsConnecting) {
    let result = await wait(() => ws && ws.identified, 3000)
    if(!result)
      return await authorizeGateway()
    else
      return ws
  } else if (ws && ws.readyState == 1 && ws.identified) {
    return ws
  }

  wsConnecting = true
  try {
    ws = new WebSocket((await gatewayUrl()).url)
    ws.identified = false
  } catch (e) {
    console.log('Authorize error', e.message)
    ws = false
    wsConnecting = false
    return
  }
  ws.on('open', gatewayOpen)
  ws.on('message', gatewayMessage.bind(null, ws, authorizeGateway, interactionResponse))
  ws.on('close', gatewayClose.bind(null, ws, authorizeGateway))
  await wait(() => ws.identified, 3000)
  wsConnecting = false
  return ws
}

function closeGateway() {
  gatewayClose(ws)
}

let previousRequest = 0

async function requestAuthQ(outgoing) {
  await authorizeGateway()
  if(typeof outgoing.headers == 'undefined')
    outgoing.headers = {}
  outgoing.headers['Authorization'] = `Bot ${TOKEN}`
  outgoing.url = DEFAULT_API + outgoing.url
  previousRequest = await delay(previousRequest, DEFAULT_RATE)
  let resolveRequest
  resolveRequest = async () => {
    const result
    try {
      //console.log('Discord request', outgoing)
      result = (await request(outgoing)).data
    } catch (e) {
      // check result for rate limit and re-run this request in a queue
      if(e.code == 429) {
        console.log('Delayed request', e.response.data.retry_after)
        await timeout(e.response.data.retry_after * 1000)
        return await resolveRequest()
      } else {
        console.log(e)
        if(e.response) {
          console.log(e.response.data.errors)
        }
        throw e
      }
    }
    return result
  }
  return await resolveRequest()
}

async function interactionResponse(interactionId, interactionToken) {
  const json = {
    'type': 5
  }
  return await requestAuthQ({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: `interactions/${interactionId}/${interactionToken}/callback`,
    data: JSON.stringify(json)
  })
}

module.exports = {
  authorizeGateway,
  gatewayUrl,
  closeGateway,
  requestAuthQ,
  interactionResponse,
}





### Discord Gateway

Responsible for maintaining connection with heartbeat. Receives and calls out gateway messages to other services.



#### the code

discord gateway?


In [ ]:
const {TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
const systemUsage = importer.import('system usage report')

const INSTANCES = {}
const SESSIONS = {}
let indentifyTimer
let privateChannels = {}
let interactions = {}
const interactionsCommands = {}
let cancelConnection // if gateway doesn't respond properly, close web socket
let heartbeat
let resources
let seq = 0

function sendHeartbeat(ws, reconnect) {
  if(!ws) return
  console.log('Sending heartbeat')
  ws.send(JSON.stringify({
    op: 1,
    d: seq
  }))
  cancelConnection = setTimeout(gatewayClose.bind(null, ws, reconnect), 4000)
}

function gatewayMessage(ws, reconnectGateway, interactionResponse, message) {
  let msgBuff = new Buffer.from(message)
  let gateway = JSON.parse(msgBuff.toString('utf-8'))
  if(gateway.s) seq = gateway.s
  if(gateway.d && gateway.d.seq) seq = gateway.d.seq
  
  console.log('Gateway message', gateway)
  
  if(gateway.op == 10) {
    ws.identified = true
    heartbeat = setInterval(sendHeartbeat.bind(null, ws, reconnectGateway), gateway.d.heartbeat_interval)
    resources = setInterval(systemUsage, 1000)
    ws.send(JSON.stringify({
      op: 2,
      intents: [
        'DIRECT_MESSAGES', 'DIRECT_MESSAGE_REACTIONS', 
        'GUILD_MESSAGES', 'GUILD_MESSAGE_REACTIONS', 
        'GUILDS', 'THREAD_UPDATE', 'THREAD_CREATE',
        'THREAD_DELETE', 'THREAD_LIST_SYNC', 'THREAD_MEMBER_UPDATE',
        'THREAD_MEMBERS_UPDATE', 'MESSAGE_CREATE', 'MESSAGE_UPDATE',
        'GUILD_PRESENCES',
      ],
      d: {
        token: TOKEN,
        properties: {
          "$os": "linux",
          "$browser": "nodejs",
          "$device": "quake3"
        }
      }
    }))
    return
  } else if (gateway.op === 7) { // should reconnect
    console.log('Reconnecting...')
    gatewayClose(ws, reconnectGateway)
    return
  } else if (gateway.op === 0 || gateway.op === 9) {
    if(gateway.t == 'MESSAGE_CREATE') {
      // guild ID can only be null if it is a personal message
      if(typeof gateway.d.guild_id == 'undefined') {
        privateChannels[gateway.d.channel_id] = Date.now()
        if(gateway.d.author.id != DEFAULT_APPLICATION) {
          interactionsCommands['promptPrivate'](gateway.d)
        }
      } else if(gateway.d.content.includes('@' + DEFAULT_APPLICATION)
        && typeof interactionsCommands['promptMention'] != 'undefined'
      ) {
        interactionsCommands['promptMention'](gateway.d)
      }
    }
    if(gateway.t == 'INTERACTION_CREATE') {
      if(typeof interactions[gateway.d.channel_id] == 'undefined')
        interactions[gateway.d.channel_id] = []
      interactions[gateway.d.channel_id].push(gateway.d)
      if(typeof interactionsCommands[gateway.d.data.name] != 'undefined') {
        Promise.resolve(interactionsCommands[gateway.d.data.name](gateway.d))
      }
      interactionResponse(gateway.d.id, gateway.d.token)
    }
    if(gateway.t == 'EMBEDDED_ACTIVITY_UPDATE_V2') {
      if(typeof INSTANCES[gateway.d['instance_id']] == 'undefined') {
        INSTANCES[gateway.d['instance_id']] = []
      }
      INSTANCES[gateway.d['instance_id']] = INSTANCES[gateway.d['instance_id']].filter(i => gateway.d.participants.includes(i[0]))
    
      if(typeof interactionsCommands['startActivity'] != 'undefined'
        && typeof interactionsCommands['endActivity'] != 'undefined'
      ) {
        if(gateway.d.participants.length) {
          Promise.resolve(interactionsCommands['startActivity'](gateway.d))
        } else {
          Promise.resolve(interactionsCommands['endActivity'](gateway.d))
        }
      }
    }
    return
  } else if (gateway.op === 11) {
    clearTimeout(cancelConnection)
    return
  }
  console.log('Unrecognized gateway', gateway)
}

function gatewayClose(ws, reconnect) {
  console.log('Discord disconnected')
  if(indentifyTimer)
    clearInterval(indentifyTimer)
  if(heartbeat)
    clearInterval(heartbeat)
  if(resources)
    clearInterval(resources)
  if(ws.readyState == 1)
    ws.close()
  if(reconnect) {
    setTimeout(reconnect, 1000)
  }
  if(typeof interactionsCommands['endActivity'] != 'undefined') {
    Promise.resolve(interactionsCommands['endActivity']())
  }
  ws.identified = false
  return
}

module.exports = {
  gatewayClose,
  gatewayMessage,
  privateChannels,
  interactions,
  interactionsCommands,
  INSTANCES,
  SESSIONS,
}




## Configuration


#### the code

discord bot configuration?


In [ ]:
let fs = require('fs')
let path = require('path')

let DEFAULT_GUILD = process.env.DEFAULT_GUILD || '319817668117135362'
let DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || '1328142980967563337' // 366715821654933515
let DEFAULT_APPLICATION = process.env.DEFAULT_APPLICATION || '1328141471840206899'
let DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v10/'
let MESSAGE_TIME = process.env.DEFAULT_TIME || 1000 * 60 * 2 // * 60 // 2 minutes to respond
let DEFAULT_RATE = 1000 / 50 // from discord documentation
let PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
let DEFAULT_USERNAME = 'Orbb'
let MESSAGES_START = 1420070400000

let tokenPath
if(fs.existsSync('./discord-bot.txt')) {
  tokenPath = path.resolve('./discord-bot.txt')
} else {
  tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}

let secretPath
if(fs.existsSync('./discord.txt')) {
  secretPath = path.resolve('./discord.txt')
} else {
  secretPath = path.join(PROFILE_PATH, '.credentials/discord.txt')
}


let TOKEN = process.env.TOKEN
if(!TOKEN && fs.existsSync(process.env.TOKENPATH || tokenPath)) {
  TOKEN = fs.readFileSync(process.env.TOKENPATH || tokenPath).toString('utf-8').trim()
}

let SECRET = process.env.SECRET
if(!SECRET && fs.existsSync(process.env.SECRETPATH || secretPath)) {
  SECRET = fs.readFileSync(process.env.SECRETPATH || secretPath).toString('utf-8').trim()
}

module.exports = {
  DEFAULT_GUILD,
  DEFAULT_CHANNEL,
  DEFAULT_APPLICATION,
  DEFAULT_API,
  MESSAGE_TIME,
  DEFAULT_RATE,
  PROFILE_PATH,
  TOKEN,
  SECRET,
  DEFAULT_USERNAME,
  MESSAGES_START
}



## Index

index.js



#### the code

discord api?


In [ ]:
let {DEFAULT_CHANNEL, DEFAULT_USERNAME} = importer.import('discord configuration')
let {
  authorizeGateway, authorizeUrl, closeGateway
} = importer.import('discord authorize')
const {requestAuthQ, interactionResponse} = importer.import('discord request')

async function triggerTyping(channelId = DEFAULT_CHANNEL) {
  return await requestAuthQ({
    method: 'POST',
    url: `channels/${channelId}/typing`
  })
}

module.exports = {
  DEFAULT_USERNAME,
  authorizeGateway,
  authorizeUrl,
  closeGateway,
  triggerTyping,
  request: requestAuthQ,
  interactionResponse,
  ... {
    userChannels, guildChannels, channelMessages, deleteChannel
  } = importer.import('discord channels'),
  ... {
    userGuilds, userConnections, getGuildRoles
  } = importer.import('discord guilds'),
  ... {
    createMessage, updateMessage,
    getPins, pinMessage, unpinMessage
  } = importer.import('discord messages'),
  ... {
    registerCommand, getCommands, deleteCommand,
    updateInteraction, updateCommand,
  } = importer.import('discord commands'),
  ... {
    createThread, archivedThreads, activeThreads,
    addThreadMember,
  } = importer.import('discord threads'),
  ... {
    getUser
  } = importer.import('discord users'),
}





## Discord REST




### Discord Message API

All of these are pretty generic just links to the route with the parameters passed into the right positions in the request object.



#### the code


discord messages?


In [ ]:

let {DEFAULT_CHANNEL} = importer.import('discord configuration')
const {requestAuthQ} = importer.import('discord request')

async function createMessage(message, channelId = DEFAULT_CHANNEL) {
  let params = typeof message == 'string' ? ({
    'content': message
  }) : message
  return await requestAuthQ({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: `channels/${channelId}/messages`,
    data: JSON.stringify(params)
  })
}

async function deleteMessage(messageId, channelId = DEFAULT_CHANNEL) {
  return await requestAuthQ({
    method: 'DELETE',
    url: `channels/${channelId}/messages/${messageId}`
  })
}

async function updateMessage(message, messageId, channelId = DEFAULT_CHANNEL) {
  let params = typeof message == 'string' ? ({
    'content': message
  }) : message
  return await requestAuthQ({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'PATCH',
    url: `channels/${channelId}/messages/${messageId}`,
    data: JSON.stringify(params)
  })
}

async function getPins(channelId = DEFAULT_CHANNEL) {
  return await requestAuthQ({
    method: 'GET',
    url: `channels/${channelId}/pins`
  })
}

async function pinMessage(messageId, channelId = DEFAULT_CHANNEL) {
  return await requestAuthQ({
    method: 'PUT',
    url: `channels/${channelId}/pins/${messageId}`
  })
}

async function unpinMessage(messageId, channelId = DEFAULT_CHANNEL) {
  return await requestAuthQ({
    method: 'DELETE',
    url: `channels/${channelId}/pins/${messageId}`
  })
}

module.exports = {
  createMessage,
  deleteMessage,
  updateMessage,
  getPins,
  pinMessage,
  unpinMessage
}





### Channels



discord channels?



In [ ]:
let {
  DEFAULT_GUILD, DEFAULT_CHANNEL, MESSAGE_TIME,
  MESSAGES_START, 
} = importer.import('discord configuration')
const {requestAuthQ} = importer.import('discord request')

async function userChannels(userId = '@me') {
  return await requestAuthQ({
    method: 'GET',
    url: `channels/${userId}`
  })
}

async function guildChannels(guildId = DEFAULT_GUILD) {
  return await requestAuthQ({
    method: 'GET',
    url: `guilds/${guildId}/channels`
  })
}

async function channelMessagesB(channelId = DEFAULT_CHANNEL, messageTime = MESSAGE_TIME) {
  let params = {
    limit: 100,
    after: messageTime.toString()
  };
  let messages = await requestAuthQ({
    method: 'GET',
    url: `channels/${channelId}/messages`,
    params
  })
  if(messages.length == 100) {
    messages = messages.concat(await channelMessagesB(channelId, BigInt(messages[0].id) + BigInt(1)))
  }
  return messages
}

async function channelMessages(channelId = DEFAULT_CHANNEL, messageTime = MESSAGE_TIME) {
  let params = {
    limit: 100,
    after: (BigInt(Date.now() - MESSAGES_START - messageTime) << BigInt(22)).toString()
  };
  let messages = await requestAuthQ({
    method: 'GET',
    url: `channels/${channelId}/messages`,
    params
  })
  if(messages.length == 100) {
    messages = messages.concat(await channelMessagesB(channelId, BigInt(messages[0].id) + BigInt(1)))
  }
  return messages
}

async function deleteChannel(channelId) {
  return await requestAuthQ({
    method: 'DELETE',
    url: `channels/${channelId}`
  })
}

module.exports = {
  userChannels,
  guildChannels,
  channelMessages,
  deleteChannel
}




### Commands



discord commands?


In [ ]:

const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const {timeout} = importer.import('discord utilities')
const {requestAuthQ} = importer.import('discord request')


async function registerCommand(cmd, desc, guildId = null) {
  // TODO: guild specific commands
  //url = "https://discord.com/api/v8/applications/<my_application_id>/guilds/<guild_id>/commands"
  let json
  if(typeof cmd == 'object') {
    json = cmd
  } else {
    json = {
      'name': cmd,
      'description': desc
    }
  }
  console.log('Registering command ', json.name)
  await timeout(2000)
  return await requestAuthQ({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands`
      : `applications/${DEFAULT_APPLICATION}/commands`,
    data: JSON.stringify(json)
  })
}

async function getCommands(guildId = null) {
  return await requestAuthQ({
    method: 'GET',
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands`
      : `applications/${DEFAULT_APPLICATION}/commands`
  })
}

async function getCommand(commandId, guildId = null) {
  return await requestAuthQ({
    method: 'GET',
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands/${commandId}`
      : `applications/${DEFAULT_APPLICATION}/commands/${commandId}`
  })
}

async function updateInteraction(message, interactionId, interactionToken) {
  let json = typeof message == 'string' ? ({
      'content': message
    }) : message
  return await requestAuthQ({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'PATCH',
    url: `webhooks/${DEFAULT_APPLICATION}/${interactionToken}/messages/@original`,
    data: JSON.stringify(json)
  })
}

async function updateCommand(cmd, desc, commandId, guildId = null) {
  if(typeof cmd == 'object') {
    json = cmd
  } else {
    json = {
      'name': cmd,
      'description': desc
    }
  }
  console.log('Updating command ', json.name)
  await timeout(2000)
  return await requestAuthQ({
    method: 'PATCH',
    headers: {
      'Content-Type': 'application/json'
    },
    url: guildId
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands/${commandId}`
      : `applications/${DEFAULT_APPLICATION}/commands/${commandId}`,
    data: JSON.stringify(json)
  })
}

async function deleteCommand(commandId, guildId = null) {
  console.log('Deleting command ', commandId)
  return await requestAuthQ({
    method: 'DELETE',
    url: guildId 
      ? `applications/${DEFAULT_APPLICATION}/guilds/${guildId}/commands/${commandId}`
      : `applications/${DEFAULT_APPLICATION}/commands/${commandId}`
  })
}

module.exports = {
  registerCommand,
  getCommands,
  getCommand,
  updateInteraction,
  deleteCommand,
  updateCommand,
}



### Guilds



discord guilds?


In [ ]:
let {DEFAULT_GUILD} = importer.import('discord configuration')
let {request} = importer.import('discord authorization')


async function userGuilds(userId = '@me') {
  return await request({
    method: 'GET',
    url: `users/${userId}/guilds`
  })
}

async function getGuildRoles(guildId = DEFAULT_GUILD) {
  return await request({
    method: 'GET',
    url: `guilds/${guildId}/roles`
  })
}

async function userConnections(userId = '@me') {
  return await request({
    method: 'GET',
    url: `users/${userId}/connections`
  })
}

module.exports = {
  userGuilds,
  getGuildRoles,
  userConnections
}



### Threads



discord threads?


In [ ]:
let {DEFAULT_CHANNEL} = importer.import('discord configuration')
const {requestAuthQ} = importer.import('discord request')

async function createThread(name, channelId = DEFAULT_CHANNEL) {
  let json = {
    'name': name,
    'type': 11,
    'auto_archive_duration': 60
  }
  return await requestAuthQ({
    headers: {
      'Content-Type': 'application/json'
    },
    method: 'POST',
    url: `channels/${channelId}/threads`,
    data: JSON.stringify(json)
  })
}

async function archivedThreads(channelId = DEFAULT_CHANNEL) {
  return await requestAuthQ({
    method: 'GET',
    url: `channels/${channelId}/threads/archived/public`
  })
}

async function activeThreads(channelId = DEFAULT_CHANNEL) {
  return await requestAuthQ({
    method: 'GET',
    url: `channels/${channelId}/threads/active`
  })
}

async function addThreadMember(memberId, channelId) {
  return await requestAuthQ({
    method: 'PUT',
    url: `/channels/${channelId}/thread-members/${memberId}`
  })
}

module.exports = {
  createThread,
  archivedThreads,
  activeThreads,
  addThreadMember,
}



### Users



discord users?


In [ ]:
const {requestAuthQ} = importer.import('discord request')

async function getUser(userId = '@me') {
  return await requestAuthQ({
    method: 'GET',
    url: `/users/${userId}`
  })
}

module.exports = {
  getUser,
}



## Utilities



### Wait and timeout



discord utilities?


In [ ]:
let timers = {}
let mainTimer = setInterval(callResolve, 20)

function callResolve() {
  let now = Date.now()
  let times = Object.keys(timers)
  for(let i = 0; i < times.length; i++) {
    if(now > times[i]) {
      try {
        Promise.resolve(timers[times[i]]())
      } catch (e) {
        console.log('timer failed', e)
        throw e
      }
      delete timers[times[i]]
      return
    }
  }
}

function addResolve(resolve, time) {
  while(typeof timers[time] != 'undefined') {
    time++
  }
  timers[time] = resolve
}

async function timeout(delay) {
  let now = Date.now()
  await new Promise(resolve => addResolve(resolve, now + delay))  
}

async function delay(prev, delay) {
  let now = Date.now()
  if(now - prev < delay)
    await new Promise(resolve => addResolve(resolve, now + (delay - (now - prev))))
  return Date.now()
}

async function wait(until, delay) {
  let waitTimer
  let waitCount = 0
  let result
  let now = Date.now()
  let delayed = now + delay
  while(!result && now < delayed) {
    await timeout(100)
    result = await until()
    now = Date.now()
  }
  return result
}

module.exports = {
  timeout,
  delay,
  wait,
}



### Delete all commands



delete all commands?


In [ ]:
const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {timeout} = importer.import('discord utilities')

const EXCEPT_COMMANDS = [
  
]

async function deleteCommands(guildId) {
  let toRemove = await getCommands(guildId)
  for(let i = 0; i < toRemove.length; i++) {
    if(EXCEPT_COMMANDS.includes(toRemove[i].name))
      continue
    await timeout(3000)
    await deleteCommand(toRemove[i].id, guildId)
  }
}


module.exports = deleteCommands

